# Estimate Continuous-time outcome measures
This `R` notebook performs hazard rate regressions to estimate the predicted outcomes of the game that are measured continuously as users take actions.

In [1]:
#install.packages("plyr")
library(plyr)
library(survival)

In [2]:
    working_dir = "../results-anonymized/experiment/"
filenames <- Sys.glob(paste(working_dir,"block_*_hazards.csv", sep=""))
filenames

[1] "../results-anonymized/experiment/block_0_hazards.csv" 
 [2] "../results-anonymized/experiment/block_10_hazards.csv"
 [3] "../results-anonymized/experiment/block_11_hazards.csv"
 [4] "../results-anonymized/experiment/block_12_hazards.csv"
 [5] "../results-anonymized/experiment/block_13_hazards.csv"
 [6] "../results-anonymized/experiment/block_14_hazards.csv"
 [7] "../results-anonymized/experiment/block_15_hazards.csv"
 [8] "../results-anonymized/experiment/block_16_hazards.csv"
 [9] "../results-anonymized/experiment/block_17_hazards.csv"
[10] "../results-anonymized/experiment/block_18_hazards.csv"
[11] "../results-anonymized/experiment/block_19_hazards.csv"
[12] "../results-anonymized/experiment/block_1_hazards.csv" 
[13] "../results-anonymized/experiment/block_20_hazards.csv"
[14] "../results-anonymized/experiment/block_21_hazards.csv"
[15] "../results-anonymized/experiment/block_22_hazards.csv"
[16] "../results-anonymized/experiment/block_23_hazards.csv"
[17] "../results-anonymized/experiment/block_24_hazards.csv"
[18] "../results-anonymized/experiment/block_25_hazards.csv"
[19] "../results-anonymized/experiment/block_26_hazards.csv"
[20] "../results-anonymized/experiment/block_27_hazards.csv"
[21] "../results-anonymized/experiment/block_28_hazards.csv"
[22] "../results-anonymized/experiment/block_29_hazards.csv"
[23] "../results-anonymized/experiment/block_2_hazards.csv" 
[24] "../results-anonymized/experiment/block_3_hazards.csv" 
[25] "../results-anonymized/experiment/block_4_hazards.csv" 
[26] "../results-anonymized/experiment/block_5_hazards.csv" 
[27] "../results-anonymized/experiment/block_6_hazards.csv" 
[28] "../results-anonymized/experiment/block_7_hazards.csv" 
[29] "../results-anonymized/experiment/block_8_hazards.csv" 
[30] "../results-anonymized/experiment/block_9_hazards.csv"

In [3]:
# load the total dataset
df <- ldply(lapply(filenames, read.csv, header=TRUE, stringsAsFactors=TRUE), rbind)
names(df)

[1] "X"                      "start"                  "exposure_id"           
 [4] "player_id"              "game_id"                "is_treatment_condition"
 [7] "is_spoke"               "is_link_or_spur"        "is_prompt"             
[10] "is_in_leads"            "is_in_deads"            "n_exposures"           
[13] "n_existing_leads"       "in_startup"             "in_peak"               
[16] "in_tail"                "in_close"               "n_rim_connections"     
[19] "n_triangle_paths"       "n_shared_edges"         "is_link"               
[22] "is_spur"                "is_exposed_t0"          "is_held_t0"            
[25] "stop"                   "adopt_event"            "forget_event"          
[28] "is_caveman_game"

In [4]:
# create subset that only includes clues that are candidates for adoption
# also, only look at spoke clues which are the same across conditions
# and ignore the clues an individual is exposed to at t0, because their neighbors didn't choose to adopt them
adopt_ht = subset(df, is_in_leads == 0 & n_exposures > 0 & is_spoke == 1 & is_exposed_t0==0)
dim(adopt_ht)

[1] 187503     28

In [5]:
# create interaction terms with treatment
adopt_ht$t_n_exposures = adopt_ht$n_exposures * adopt_ht$is_treatment_condition
adopt_ht$t_n_rim_connections = adopt_ht$n_rim_connections * adopt_ht$is_treatment_condition
adopt_ht$t_is_in_deads = adopt_ht$is_in_deads * adopt_ht$is_treatment_condition

adopt_ht$t_n_shared_edges <- adopt_ht$n_shared_edges * adopt_ht$is_treatment_condition
adopt_ht$X <- NULL
rownames(adopt_ht) <- NULL  # reindex
tail(adopt_ht)

,start,exposure_id,player_id,game_id,is_treatment_condition,is_spoke,is_link_or_spur,is_prompt,is_in_leads,is_in_deads,⋯,is_exposed_t0,is_held_t0,stop,adopt_event,forget_event,is_caveman_game,t_n_exposures,t_n_rim_connections,t_is_in_deads,t_n_shared_edges
,<dbl>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>
187498,177.287,zvbyhJXXEfmaYzExx_tclue_1_3,zvbyhJXXEfmaYzExx,Lj2up2ePZA6WtiK4q,1,1,0,0,0,0,⋯,0,0,180.390,0,0,0,1,0,0,3
187499,180.390,zvbyhJXXEfmaYzExx_tclue_1_3,zvbyhJXXEfmaYzExx,Lj2up2ePZA6WtiK4q,1,1,0,0,0,0,⋯,0,0,192.021,0,0,0,1,0,0,3
187500,192.021,zvbyhJXXEfmaYzExx_tclue_1_3,zvbyhJXXEfmaYzExx,Lj2up2ePZA6WtiK4q,1,1,0,0,0,0,⋯,0,0,215.988,0,0,0,1,0,0,3
187501,215.988,zvbyhJXXEfmaYzExx_tclue_1_3,zvbyhJXXEfmaYzExx,Lj2up2ePZA6WtiK4q,1,1,0,0,0,0,⋯,0,0,274.276,0,0,0,1,0,0,3
187502,274.276,zvbyhJXXEfmaYzExx_tclue_1_3,zvbyhJXXEfmaYzExx,Lj2up2ePZA6WtiK4q,1,1,0,0,0,0,⋯,0,0,344.175,0,0,0,2,0,0,5
187503,344.175,zvbyhJXXEfmaYzExx_tclue_1_3,zvbyhJXXEfmaYzExx,Lj2up2ePZA6WtiK4q,1,1,0,0,0,0,⋯,0,0,371.758,1,0,0,2,0,0,5


# Estimate effect of familiarity, supporting pathways on adoption

Use this regression in the manipulation check - if there is no effect of n_triangle_paths, then we expect the treatment and control conditions to be identical, as there isn't interdependence between clues other than through the familiarity mechanism, and so the spoke clues are independent of one another.

In [6]:
factors_fit <- coxph(
    Surv(time=start, time2=stop, event=adopt_event, type='counting') ~
        in_startup + in_peak + in_close + in_tail + # timing dummies 
        n_exposures + t_n_exposures + # social reinforcement
        n_triangle_paths +  # "logical" support
        n_rim_connections + t_n_rim_connections + # familiarity with concepts
        is_in_deads + t_is_in_deads + # prior rejection
        n_existing_leads +  # current state of notebook
        frailty.gaussian(player_id),  # random player effect (as players have different play speeds)
    data=adopt_ht,
)

factors_fit

Warning message in Surv(time = start, time2 = stop, event = adopt_event, type = "counting"):
“Stop time must be > start time, NA created”


Call:
coxph(formula = Surv(time = start, time2 = stop, event = adopt_event, 
    type = "counting") ~ in_startup + in_peak + in_close + in_tail + 
    n_exposures + t_n_exposures + n_triangle_paths + n_rim_connections + 
    t_n_rim_connections + is_in_deads + t_is_in_deads + n_existing_leads + 
    frailty.gaussian(player_id), data = adopt_ht)

                               coef  se(coef)       se2     Chisq   DF      p
in_startup                -1.42e+00  2.14e+00  2.14e+00  4.38e-01    1  0.508
in_peak                   -4.04e-01  4.44e-01  4.44e-01  8.31e-01    1  0.362
in_close                  -3.73e-01  4.04e-01  4.04e-01  8.53e-01    1  0.356
in_tail                              0.00e+00  0.00e+00              1       
n_exposures                3.76e-01  2.62e-02  2.43e-02  2.06e+02    1 <2e-16
t_n_exposures             -7.86e-02  3.17e-02  2.52e-02  6.15e+00    1  0.013
n_triangle_paths           4.33e-02  2.00e-02  1.95e-02  4.70e+00    1  0.030
n_rim_connections          4

In [7]:
factors_summary = summary(factors_fit)
factors_coefs = merge(factors_summary$conf.int, factors_summary$coefficients, by='row.names')
rownames(factors_coefs) <- factors_coefs$Row.names  # reset row names
factors_coefs$Row.names <- NULL

# add means
factors_coefs$means <- factors_fit$means[row.names(factors_coefs)]

# export data for plotting in python
write.csv(factors_coefs, paste(working_dir, "factors_cox_results.csv", sep=""))
factors_coefs

,exp(coef),exp(-coef),lower .95,upper .95,coef,se(coef),se2,Chisq,DF,p,means
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
in_close,0.68870150,1.4520079,0.312142428,1.51952989,-0.37294735,0.403756577,0.403524543,0.8532098,1,3.556459e-01,0.1590234
in_peak,0.66732322,1.4985242,0.279699030,1.59214096,-0.40448076,0.443661411,0.443568752,0.8311749,1,3.619330e-01,0.2646389
in_startup,0.24225474,4.1278861,0.003634427,16.14762593,-1.41776545,2.142661048,2.142380957,0.4378263,1,5.081735e-01,0.0074308
in_tail,NA,NA,NA,NA,NA,0.000000000,0.000000000,NA,1,NA,0.5689069
is_in_deads,0.08601561,11.6257971,0.075774385,0.09764097,-2.45322652,0.064678969,0.063552939,1438.6300320,1,8.474653e-315,0.2260393
n_existing_leads,1.06260833,0.9410805,1.056380327,1.06887304,0.06072657,0.002999184,0.002563618,409.9693567,1,3.722175e-91,11.5020084
n_exposures,1.45637475,0.6866365,1.383407253,1.53319090,0.37595030,0.026225390,0.024306794,205.5024346,1,1.315699e-46,1.2527645
n_rim_connections,1.54873531,0.6456881,1.515255194,1.58295518,0.43743867,0.011150613,0.010864900,1538.9938545,1,0.000000e+00,0.9522306
n_triangle_paths,1.04425817,0.9576176,1.004144346,1.08597448,0.04330675,0.019985556,0.019476369,4.6954666,1,3.024229e-02,0.2630226


# Estimate effect of similarity to exposers

The theoretical argument is that logic and familiarity increase polarization by making individuals imitate their more similar alters. So, we swap out the logic and familiarity regressors with the similarity measure

In [8]:
similarity_fit <- coxph(
    Surv(time=start, time2=stop, event=adopt_event, type='counting') ~
        in_startup + in_peak + in_close + in_tail +  # timing dummies 
        n_exposures + t_n_exposures +  # social reinforcement
        n_shared_edges + t_n_shared_edges +  # similarity to exposers
        is_in_deads + t_is_in_deads +  # prior rejection
        n_existing_leads +  # current state of notebook
        frailty.gaussian(player_id),  # random player effect (as players have different play speeds)
    data=adopt_ht,
)


similarity_fit

Warning message in Surv(time = start, time2 = stop, event = adopt_event, type = "counting"):
“Stop time must be > start time, NA created”


Call:
coxph(formula = Surv(time = start, time2 = stop, event = adopt_event, 
    type = "counting") ~ in_startup + in_peak + in_close + in_tail + 
    n_exposures + t_n_exposures + n_shared_edges + t_n_shared_edges + 
    is_in_deads + t_is_in_deads + n_existing_leads + frailty.gaussian(player_id), 
    data = adopt_ht)

                               coef  se(coef)       se2     Chisq   DF       p
in_startup                -1.45e+00  2.15e+00  2.15e+00  4.54e-01    1    0.50
in_peak                   -4.14e-01  4.42e-01  4.42e-01  8.77e-01    1    0.35
in_close                  -4.04e-01  4.06e-01  4.06e-01  9.89e-01    1    0.32
in_tail                              0.00e+00  0.00e+00              1        
n_exposures                4.16e-01  2.88e-02  2.70e-02  2.08e+02    1 < 2e-16
t_n_exposures              6.93e-03  3.55e-02  2.92e-02  3.81e-02    1    0.85
n_shared_edges             2.96e-03  4.47e-03  4.27e-03  4.38e-01    1    0.51
t_n_shared_edges          -2.72e-02  6.00e-03

In [9]:
similarity_summary = summary(similarity_fit)
similarity_coefs = merge(similarity_summary$conf.int, similarity_summary$coefficients, by='row.names')
rownames(similarity_coefs) <- similarity_coefs$Row.names  # reset row names
similarity_coefs$Row.names <- NULL

# add means
similarity_coefs$means <- similarity_fit$means[row.names(similarity_coefs)]

# export data for plotting in python
write.csv(similarity_coefs, paste(working_dir, "similarity_cox_results.csv", sep=""))
similarity_coefs

,exp(coef),exp(-coef),lower .95,upper .95,coef,se(coef),se2,Chisq,DF,p,means
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
in_close,0.66754786,1.4980199,0.300942730,1.48074732,-0.404144198,0.406482521,0.406311243,9.885279e-01,1,3.201024e-01,0.1590234
in_peak,0.66076680,1.5133932,0.277656785,1.57249088,-0.414354302,0.442362828,0.442273125,8.773774e-01,1,3.489210e-01,0.2646389
in_startup,0.23523418,4.2510829,0.003488604,15.86167808,-1.447173749,2.148549564,2.148257010,4.536810e-01,1,5.005921e-01,0.0074308
in_tail,NA,NA,NA,NA,NA,0.000000000,0.000000000,NA,1,NA,0.5689069
is_in_deads,0.06627338,15.0890153,0.058405264,0.07520145,-2.713967012,0.064481922,0.063371580,1.771467e+03,1,0.000000e+00,0.2260393
n_existing_leads,1.10988739,0.9009923,1.102589843,1.11723323,0.104258556,0.003365746,0.002938708,9.595349e+02,1,1.122304e-210,11.5020084
n_exposures,1.51518773,0.6599842,1.432059756,1.60314109,0.415539342,0.028789073,0.027012014,2.083382e+02,1,3.165382e-47,1.2527645
n_shared_edges,1.00296599,0.9970428,0.994208886,1.01180023,0.002961601,0.004474341,0.004271862,4.381217e-01,1,5.080304e-01,5.3117148
t_is_in_deads,1.95704784,0.5109737,1.644775183,2.32860776,0.671437134,0.088692153,0.087056476,5.731131e+01,1,3.720103e-14,0.1063036


# Estimate Game-level effects for mediation analysis

In [10]:
game_treat = factor(paste(adopt_ht$game_id, adopt_ht$is_treatment_condition))
dummies <- data.frame(model.matrix( ~ game_treat))
dummies$X.Intercept. <- NULL  # drop intercept column
names(dummies) <- gsub("\\.", "_", names(dummies))
names(dummies) <- gsub("game_treat", "GT_", names(dummies))


game_n_triangle = dummies*adopt_ht$n_triangle_paths
names(game_n_triangle) <- paste(names(game_n_triangle), "_nTriangles", sep="")
game_n_shared_edges = dummies*adopt_ht$n_shared_edges
names(game_n_shared_edges) <- paste(names(game_n_shared_edges), "_nShared", sep="")


adopt_ht_dummies <- cbind(adopt_ht, game_n_triangle, game_n_shared_edges)
adopt_ht_dummies

,start,exposure_id,player_id,game_id,is_treatment_condition,is_spoke,is_link_or_spur,is_prompt,is_in_leads,is_in_deads,⋯,GT_yBcW7BjFe59HDfshF_0_nShared,GT_yBcW7BjFe59HDfshF_1_nShared,GT_yGuanMgCM54YkTWBP_0_nShared,GT_yGuanMgCM54YkTWBP_1_nShared,GT_Zae6heWeMutojmXft_0_nShared,GT_Zae6heWeMutojmXft_1_nShared,GT_zg5puDzKGCxk7wAj2_0_nShared,GT_zg5puDzKGCxk7wAj2_1_nShared,GT_Zxhtd5NFdLyNExLtN_0_nShared,GT_Zxhtd5NFdLyNExLtN_1_nShared
,<dbl>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,66.137,2xywv6ujoSmDKZYW3_cclue_1_10,2xywv6ujoSmDKZYW3,nkPob7gypQi6DtwqS,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,70.877,2xywv6ujoSmDKZYW3_cclue_1_10,2xywv6ujoSmDKZYW3,nkPob7gypQi6DtwqS,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,72.379,2xywv6ujoSmDKZYW3_cclue_1_10,2xywv6ujoSmDKZYW3,nkPob7gypQi6DtwqS,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,73.710,2xywv6ujoSmDKZYW3_cclue_1_10,2xywv6ujoSmDKZYW3,nkPob7gypQi6DtwqS,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,74.747,2xywv6ujoSmDKZYW3_cclue_1_10,2xywv6ujoSmDKZYW3,nkPob7gypQi6DtwqS,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,75.658,2xywv6ujoSmDKZYW3_cclue_1_10,2xywv6ujoSmDKZYW3,nkPob7gypQi6DtwqS,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
7,76.503,2xywv6ujoSmDKZYW3_cclue_1_10,2xywv6ujoSmDKZYW3,nkPob7gypQi6DtwqS,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8,78.462,2xywv6ujoSmDKZYW3_cclue_1_10,2xywv6ujoSmDKZYW3,nkPob7gypQi6DtwqS,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
9,78.532,2xywv6ujoSmDKZYW3_cclue_1_10,2xywv6ujoSmDKZYW3,nkPob7gypQi6DtwqS,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [11]:
names(game_n_shared_edges)

[1] "GT_2RErnuyZ5YmzTzpxh_1_nShared" "GT_55mC2NYGpS5sYb46x_0_nShared"
  [3] "GT_55mC2NYGpS5sYb46x_1_nShared" "GT_7c9biMCakuPCxMEXN_0_nShared"
  [5] "GT_7c9biMCakuPCxMEXN_1_nShared" "GT_7kcXpJaQqJmx68nYz_0_nShared"
  [7] "GT_7kcXpJaQqJmx68nYz_1_nShared" "GT_9oijjTA5XEFzCtedK_0_nShared"
  [9] "GT_9oijjTA5XEFzCtedK_1_nShared" "GT_aS5rH5N5tnRsr3CQQ_0_nShared"
 [11] "GT_aS5rH5N5tnRsr3CQQ_1_nShared" "GT_b4CBPunda96KnrhFi_0_nShared"
 [13] "GT_b4CBPunda96KnrhFi_1_nShared" "GT_BfWPcLgXY7dKcAsdX_0_nShared"
 [15] "GT_BfWPcLgXY7dKcAsdX_1_nShared" "GT_BfyZ5JjRKWhdjS7qy_0_nShared"
 [17] "GT_BfyZ5JjRKWhdjS7qy_1_nShared" "GT_BhccgcKQe7bpH7pWi_0_nShared"
 [19] "GT_BhccgcKQe7bpH7pWi_1_nShared" "GT_c2BKscFKSBaujzQHw_0_nShared"
 [21] "GT_c2BKscFKSBaujzQHw_1_nShared" "GT_d2N6gipGzAM2B3GPn_0_nShared"
 [23] "GT_d2N6gipGzAM2B3GPn_1_nShared" "GT_dC7qKt8rhj8W3FFTC_0_nShared"
 [25] "GT_dC7qKt8rhj8W3FFTC_1_nShared" "GT_dW6HD8ba9gXHqoWL8_0_nShared"
 [27] "GT_dW6HD8ba9gXHqoWL8_1_nShared" "GT_e6Kq4iEPeKDdJ3wyk_0_nShared"
 [29] "GT_e6Kq4iEPeKDdJ3wyk_1_nShared" "GT_EFdcytJKJJAgFebvR_0_nShared"
 [31] "GT_EFdcytJKJJAgFebvR_1_nShared" "GT_ETYJ4Ru9uE3Fszkkr_0_nShared"
 [33] "GT_ETYJ4Ru9uE3Fszkkr_1_nShared" "GT_ew8uue7goY6SzSnMn_0_nShared"
 [35] "GT_ew8uue7goY6SzSnMn_1_nShared" "GT_F6g9NDczqP966pA5m_0_nShared"
 [37] "GT_F6g9NDczqP966pA5m_1_nShared" "GT_hE8iDmuY7wvbgTTRB_0_nShared"
 [39] "GT_hE8iDmuY7wvbgTTRB_1_nShared" "GT_hjTbDvCzT2RW9pgm8_0_nShared"
 [41] "GT_hjTbDvCzT2RW9pgm8_1_nShared" "GT_HXeLe9DnXEMs6JFjf_0_nShared"
 [43] "GT_HXeLe9DnXEMs6JFjf_1_nShared" "GT_ig8z4GCN3uW4nFT54_0_nShared"
 [45] "GT_ig8z4GCN3uW4nFT54_1_nShared" "GT_j5d27JCC2rTDBr4Yq_0_nShared"
 [47] "GT_j5d27JCC2rTDBr4Yq_1_nShared" "GT_jcfFiGC22Api8cm7o_0_nShared"
 [49] "GT_jcfFiGC22Api8cm7o_1_nShared" "GT_KC4EWWmEew2eiQFcQ_0_nShared"
 [51] "GT_KC4EWWmEew2eiQFcQ_1_nShared" "GT_KtFjB6pDKRispFwCs_0_nShared"
 [53] "GT_KtFjB6pDKRispFwCs_1_nShared" "GT_L44rB9SbzauAEuMGe_0_nShared"
 [55] "GT_L44rB9SbzauAEuMGe_1_nShared" "GT_Lj2up2ePZA6WtiK4q_0_nShared"
 [57] "GT_Lj2up2ePZA6WtiK4q_1_nShared" "GT_MfjKnfErfu8AMLKps_0_nShared"
 [59] "GT_MfjKnfErfu8AMLKps_1_nShared" "GT_mSJbtZFsaaWC3ANqy_0_nShared"
 [61] "GT_mSJbtZFsaaWC3ANqy_1_nShared" "GT_mYHZ32AyKCmNrfSY5_0_nShared"
 [63] "GT_mYHZ32AyKCmNrfSY5_1_nShared" "GT_nkPob7gypQi6DtwqS_0_nShared"
 [65] "GT_nkPob7gypQi6DtwqS_1_nShared" "GT_NkvLcmxvQY36f759u_0_nShared"
 [67] "GT_NkvLcmxvQY36f759u_1_nShared" "GT_NNqcb487RHAbLvNso_0_nShared"
 [69] "GT_NNqcb487RHAbLvNso_1_nShared" "GT_nyj44mrAdH9BGGSnK_0_nShared"
 [71] "GT_nyj44mrAdH9BGGSnK_1_nShared" "GT_oWf8QrmFrj8ZiE6JK_0_nShared"
 [73] "GT_oWf8QrmFrj8ZiE6JK_1_nShared" "GT_PRBmQFw8WDcrNi2Qn_0_nShared"
 [75] "GT_PRBmQFw8WDcrNi2Qn_1_nShared" "GT_qQyhQa275SKLzM3X9_0_nShared"
 [77] "GT_qQyhQa275SKLzM3X9_1_nShared" "GT_R5JmsFyk7arzZrwiE_0_nShared"
 [79] "GT_R5JmsFyk7arzZrwiE_1_nShared" "GT_RLNbvyv9ipkrkPJKc_0_nShared"
 [81] "GT_RLNbvyv9ipkrkPJKc_1_nShared" "GT_rtzgQPuxzH7dGbna4_0_nShared"
 [83] "GT_rtzgQPuxzH7dGbna4_1_nShared" "GT_RugWRCuQ9MounHF8x_0_nShared"
 [85] "GT_RugWRCuQ9MounHF8x_1_nShared" "GT_SamzHPveTXj44MS6F_0_nShared"
 [87] "GT_SamzHPveTXj44MS6F_1_nShared" "GT_sKStjcTJvnnofxCgB_0_nShared"
 [89] "GT_sKStjcTJvnnofxCgB_1_nShared" "GT_szPuQL8WF5prJP9s7_0_nShared"
 [91] "GT_szPuQL8WF5prJP9s7_1_nShared" "GT_T9KCzCFiibi6W5mnL_0_nShared"
 [93] "GT_T9KCzCFiibi6W5mnL_1_nShared" "GT_TCrgRvccdKGHeDvdW_0_nShared"
 [95] "GT_TCrgRvccdKGHeDvdW_1_nShared" "GT_To5RB7JuNk4h5FHaZ_0_nShared"
 [97] "GT_To5RB7JuNk4h5FHaZ_1_nShared" "GT_W3Pr3tyHAoPsyoZxh_0_nShared"
 [99] "GT_W3Pr3tyHAoPsyoZxh_1_nShared" "GT_W4EAr6TwRFApoCagP_0_nShared"
[101] "GT_W4EAr6TwRFApoCagP_1_nShared" "GT_W7ZjgcCnPfhq8eYnp_0_nShared"
[103] "GT_W7ZjgcCnPfhq8eYnp_1_nShared" "GT_wx5E4WNYAzhggxxNt_0_nShared"
[105] "GT_wx5E4WNYAzhggxxNt_1_nShared" "GT_xNo5TvYuoLwYHcgHv_0_nShared"
[107] "GT_xNo5TvYuoLwYHcgHv_1_nShared" "GT_yAmdFLN5XZJ6SxeKL_0_nShared"
[109] "GT_yAmdFLN5XZJ6SxeKL_1_nShared" "GT_yBcW7BjFe59HDfshF_0_nShared"
[111] "GT_yBcW7BjFe59HDfshF_1_nShared" "GT

In [12]:
fmla <- as.formula(paste(
    "Surv(time=start, time2=stop, event=adopt_event, type='counting') ~ 
        in_startup + in_peak + in_close + in_tail +  
        n_exposures + t_n_exposures + ",
    paste(names(game_n_triangle), collapse= "+"),
    " + n_rim_connections + t_n_rim_connections + 
        is_in_deads + t_is_in_deads + 
        n_existing_leads +  
        frailty.gaussian(player_id)"
))  
print(fmla)
 
game_factors_fit <- coxph(fmla,
    data=adopt_ht_dummies,
)

game_factors_fit

Surv(time = start, time2 = stop, event = adopt_event, type = "counting") ~ 
    in_startup + in_peak + in_close + in_tail + n_exposures + 
        t_n_exposures + GT_2RErnuyZ5YmzTzpxh_1_nTriangles + GT_55mC2NYGpS5sYb46x_0_nTriangles + 
        GT_55mC2NYGpS5sYb46x_1_nTriangles + GT_7c9biMCakuPCxMEXN_0_nTriangles + 
        GT_7c9biMCakuPCxMEXN_1_nTriangles + GT_7kcXpJaQqJmx68nYz_0_nTriangles + 
        GT_7kcXpJaQqJmx68nYz_1_nTriangles + GT_9oijjTA5XEFzCtedK_0_nTriangles + 
        GT_9oijjTA5XEFzCtedK_1_nTriangles + GT_aS5rH5N5tnRsr3CQQ_0_nTriangles + 
        GT_aS5rH5N5tnRsr3CQQ_1_nTriangles + GT_b4CBPunda96KnrhFi_0_nTriangles + 
        GT_b4CBPunda96KnrhFi_1_nTriangles + GT_BfWPcLgXY7dKcAsdX_0_nTriangles + 
        GT_BfWPcLgXY7dKcAsdX_1_nTriangles + GT_BfyZ5JjRKWhdjS7qy_0_nTriangles + 
        GT_BfyZ5JjRKWhdjS7qy_1_nTriangles + GT_BhccgcKQe7bpH7pWi_0_nTriangles + 
        GT_BhccgcKQe7bpH7pWi_1_nTriangles + GT_c2BKscFKSBaujzQHw_0_nTriangles + 
        GT_c2BKscFKSBaujzQHw_1_nTri

Warning message in Surv(time = start, time2 = stop, event = adopt_event, type = "counting"):
“Stop time must be > start time, NA created”


Call:
coxph(formula = fmla, data = adopt_ht_dummies)

                               coef  se(coef)       se2     Chisq   DF       p
in_startup                -1.40e+00  2.13e+00  2.13e+00  4.34e-01    1 0.51014
in_peak                   -4.06e-01  4.45e-01  4.45e-01  8.34e-01    1 0.36105
in_close                  -4.07e-01  4.07e-01  4.07e-01  9.99e-01    1 0.31755
in_tail                              0.00e+00  0.00e+00              1        
n_exposures                3.82e-01  2.62e-02  2.43e-02  2.13e+02    1 < 2e-16
t_n_exposures             -1.03e-01  3.14e-02  2.51e-02  1.07e+01    1 0.00104
GT_2RErnuyZ5YmzTzpxh_1_nT -2.01e-02  1.82e-01  1.60e-01  1.23e-02    1 0.91185
GT_55mC2NYGpS5sYb46x_0_nT  1.04e+00  2.81e-01  2.57e-01  1.37e+01    1 0.00021
GT_55mC2NYGpS5sYb46x_1_nT  4.37e-01  1.82e-01  1.61e-01  5.74e+00    1 0.01663
GT_7c9biMCakuPCxMEXN_0_nT -3.25e-01  2.13e-01  2.05e-01  2.33e+00    1 0.12727
GT_7c9biMCakuPCxMEXN_1_nT  3.79e-01  1.27e-01  1.21e-01  8.85e+00    1 0.0029

In [13]:
fmla <- as.formula(paste(
    "Surv(time=start, time2=stop, event=adopt_event, type='counting') ~ 
        in_startup + in_peak + in_close + in_tail +  
        n_exposures + t_n_exposures + ",
    paste(names(game_n_shared_edges), collapse= "+"),
    " + n_rim_connections + t_n_rim_connections + 
        is_in_deads + t_is_in_deads + 
        n_existing_leads +  
        frailty.gaussian(player_id)"
))  
print(fmla)
 
game_similarity_fit <- coxph(fmla,
    data=adopt_ht_dummies,
)

game_similarity_fit

Surv(time = start, time2 = stop, event = adopt_event, type = "counting") ~ 
    in_startup + in_peak + in_close + in_tail + n_exposures + 
        t_n_exposures + GT_2RErnuyZ5YmzTzpxh_1_nShared + GT_55mC2NYGpS5sYb46x_0_nShared + 
        GT_55mC2NYGpS5sYb46x_1_nShared + GT_7c9biMCakuPCxMEXN_0_nShared + 
        GT_7c9biMCakuPCxMEXN_1_nShared + GT_7kcXpJaQqJmx68nYz_0_nShared + 
        GT_7kcXpJaQqJmx68nYz_1_nShared + GT_9oijjTA5XEFzCtedK_0_nShared + 
        GT_9oijjTA5XEFzCtedK_1_nShared + GT_aS5rH5N5tnRsr3CQQ_0_nShared + 
        GT_aS5rH5N5tnRsr3CQQ_1_nShared + GT_b4CBPunda96KnrhFi_0_nShared + 
        GT_b4CBPunda96KnrhFi_1_nShared + GT_BfWPcLgXY7dKcAsdX_0_nShared + 
        GT_BfWPcLgXY7dKcAsdX_1_nShared + GT_BfyZ5JjRKWhdjS7qy_0_nShared + 
        GT_BfyZ5JjRKWhdjS7qy_1_nShared + GT_BhccgcKQe7bpH7pWi_0_nShared + 
        GT_BhccgcKQe7bpH7pWi_1_nShared + GT_c2BKscFKSBaujzQHw_0_nShared + 
        GT_c2BKscFKSBaujzQHw_1_nShared + GT_d2N6gipGzAM2B3GPn_0_nShared + 
        GT_d2N6gipGz

Warning message in Surv(time = start, time2 = stop, event = adopt_event, type = "counting"):
“Stop time must be > start time, NA created”


Call:
coxph(formula = fmla, data = adopt_ht_dummies)

                               coef  se(coef)       se2     Chisq   DF       p
in_startup                -1.43e+00  2.14e+00  2.14e+00  4.44e-01    1 0.50543
in_peak                   -4.14e-01  4.44e-01  4.44e-01  8.67e-01    1 0.35173
in_close                  -3.84e-01  4.07e-01  4.07e-01  8.91e-01    1 0.34528
in_tail                              0.00e+00  0.00e+00              1        
n_exposures                3.30e-01  2.94e-02  2.75e-02  1.26e+02    1 < 2e-16
t_n_exposures             -1.69e-03  3.69e-02  3.12e-02  2.10e-03    1 0.96347
GT_2RErnuyZ5YmzTzpxh_1_nS -6.05e-03  4.45e-02  3.70e-02  1.85e-02    1 0.89182
GT_55mC2NYGpS5sYb46x_0_nS  7.80e-02  2.56e-02  2.00e-02  9.26e+00    1 0.00234
GT_55mC2NYGpS5sYb46x_1_nS  1.22e-01  5.13e-02  4.11e-02  5.67e+00    1 0.01723
GT_7c9biMCakuPCxMEXN_0_nS -3.69e-02  1.71e-02  1.46e-02  4.66e+00    1 0.03082
GT_7c9biMCakuPCxMEXN_1_nS  2.99e-02  4.40e-02  3.76e-02  4.63e-01    1 0.4961

In [14]:
game_df_tri = data.frame(game_factors_fit$coefficients[names(game_n_triangle)])
names(game_df_tri) <- "n_triangle"
rownames(game_df_tri) <- gsub("_nTriangles", "", rownames(game_df_tri))
game_df_tri

,n_triangle
,<dbl>
GT_2RErnuyZ5YmzTzpxh_1,-0.02009542
GT_55mC2NYGpS5sYb46x_0,1.03836265
GT_55mC2NYGpS5sYb46x_1,0.43666485
GT_7c9biMCakuPCxMEXN_0,-0.32456852
GT_7c9biMCakuPCxMEXN_1,0.37912219
GT_7kcXpJaQqJmx68nYz_0,0.48024819
GT_7kcXpJaQqJmx68nYz_1,-0.02342182
GT_9oijjTA5XEFzCtedK_0,0.46599640
GT_9oijjTA5XEFzCtedK_1,0.41571107


In [15]:
game_df_sim = data.frame(game_similarity_fit$coefficients[names(game_n_shared_edges)])
names(game_df_sim) <- "n_shared_edges"
rownames(game_df_sim) <- gsub("_nShared", "", rownames(game_df_sim))


In [16]:
game_df_sim$n_triangle <- game_df_tri[rownames(game_df_sim),]
game_df_sim

,n_shared_edges,n_triangle
,<dbl>,<dbl>
GT_2RErnuyZ5YmzTzpxh_1,-0.006054214,-0.02009542
GT_55mC2NYGpS5sYb46x_0,0.078005769,1.03836265
GT_55mC2NYGpS5sYb46x_1,0.122119553,0.43666485
GT_7c9biMCakuPCxMEXN_0,-0.036888497,-0.32456852
GT_7c9biMCakuPCxMEXN_1,0.029937957,0.37912219
GT_7kcXpJaQqJmx68nYz_0,0.009262590,0.48024819
GT_7kcXpJaQqJmx68nYz_1,-0.041633252,-0.02342182
GT_9oijjTA5XEFzCtedK_0,0.047342500,0.46599640
GT_9oijjTA5XEFzCtedK_1,0.137895366,0.41571107


In [17]:
write.csv(game_df_sim, paste(working_dir, "game_level_factors.csv", sep=""))